In [1]:
from pathlib import Path
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

ON_KAGGLE = Path("/kaggle").exists()
competition_name = "playground-series-s6e2"
if ON_KAGGLE:
    sys.path.insert(0, "/kaggle/input/kaggle-utils")
    %run /kaggle/input/kaggle-utils/stat_funcs.py
    train_df = pd.read_csv(f"/kaggle/input/{competition_name}/train.csv")
    test_df  = pd.read_csv(f"/kaggle/input/{competition_name}/test.csv")
else:
    %run ../../utils/stat_funcs.py
    train_df = pd.read_csv("../../data-raw/train.csv")
    test_df  = pd.read_csv("../../data-raw/test.csv")

print("custom functions are now available in the notebook namespace!")
print("Libraries loaded successfully!")

custom functions are now available in the notebook namespace!
Libraries loaded successfully!


# Data Exploration

Target column: `Heart Disease`

Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target.

For each `id` in the test set, you must predict a probability for the `Heart Disease` target.

```csv
id,Heart Disease
630000,0.2
630001,0.3
630002,0.1
```

In [7]:
train_df.head(10)

,id,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,0,58,1,4,152,239,0,0,158,1,3.6,2,2,7,Presence
1,1,52,1,1,125,325,0,2,171,0,0.0,1,0,3,Absence
2,2,56,0,2,160,188,0,2,151,0,0.0,1,0,3,Absence
3,3,44,0,3,134,229,0,2,150,0,1.0,2,0,3,Absence
4,4,58,1,4,140,234,0,2,125,1,3.8,2,3,3,Presence
5,5,38,1,4,138,283,0,0,147,1,1.6,2,2,7,Presence
6,6,59,1,4,130,246,0,2,152,0,0.8,2,2,3,Presence
7,7,60,0,3,120,245,0,0,151,0,1.2,1,0,3,Absence
8,8,48,0,4,140,212,0,2,125,0,0.0,1,0,3,Absence
9,9,44,0,4,150,197,0,0,150,0,0.0,2,0,3,Absence


Missing values

In [3]:
train_df.isnull().sum()

id                         0
Age                        0
Sex                        0
Chest pain type            0
BP                         0
Cholesterol                0
FBS over 120               0
EKG results                0
Max HR                     0
Exercise angina            0
ST depression              0
Slope of ST                0
Number of vessels fluro    0
Thallium                   0
Heart Disease              0
dtype: int64

Data Types

In [4]:
train_df.dtypes

id                           int64
Age                          int64
Sex                          int64
Chest pain type              int64
BP                           int64
Cholesterol                  int64
FBS over 120                 int64
EKG results                  int64
Max HR                       int64
Exercise angina              int64
ST depression              float64
Slope of ST                  int64
Number of vessels fluro      int64
Thallium                     int64
Heart Disease               object
dtype: object

Number of unique values in each column

In [5]:
train_df.nunique().sort_values(ascending=False)

id                         630000
Cholesterol                   150
Max HR                         93
BP                             66
ST depression                  66
Age                            42
Chest pain type                 4
Number of vessels fluro         4
EKG results                     3
Slope of ST                     3
Thallium                        3
Sex                             2
FBS over 120                    2
Exercise angina                 2
Heart Disease                   2
dtype: int64